In [1]:
import pandas as pd
from indicators import RSI, extract_bb, MFI, CMF
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from IPython.display import clear_output
import plotly.graph_objects as go
import plotly.express as px

import gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from gym import spaces
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import ProgressBarCallback

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("gzpn_data.csv")
df = df.dropna().reset_index(drop=True)  # .sample(frac=1)
df = df.drop(columns=["<TICKER>", "<PER>", "<DATE>", "<TIME>"])
df.columns = ["open", "high", "low", "close", "volume"]
df

,open,high,low,close,volume
0,162.65,162.70,162.43,162.49,218280
1,162.49,162.50,162.31,162.42,174320
2,162.43,162.49,162.30,162.43,128060
3,162.45,162.46,162.28,162.29,125240
4,162.30,162.37,162.20,162.37,228440
...,...,...,...,...,...
2127,163.37,163.40,163.30,163.40,171700
2128,163.40,163.82,163.39,163.71,623050
2129,163.70,163.82,163.61,163.80,314660
2130,163.80,163.82,163.60,163.70,136580


In [3]:
n_steps = 14

prices = df["close"]

rsi_values = RSI(prices=prices, n_steps=n_steps)
bb_values = extract_bb(prices=prices, n_steps=n_steps)
mfi_values = MFI(
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    volume=df["volume"],
    n_steps=n_steps,
)


cmf_values = CMF(
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    volume=df["volume"],
    n_steps=n_steps,
)

assert len(rsi_values) == len(
    bb_values
), f"Indicators length don't coincide: {len(rsi_values)} and {len(bb_values)}"

In [4]:
def prepare_target(df, steps_obs: int = 3):
    targets = []
    for i in range(0, len(df) - steps_obs):
        current_price = df["close"].iloc[i]
        max_price = df["high"].iloc[i + 1 : i + 1 + steps_obs].max()
        targets.append(max_price > current_price)
    targets += [0] * steps_obs
    return np.array(targets, dtype=np.int32)


steps_obs = 3

targets = prepare_target(df=df, steps_obs=steps_obs)

In [5]:
indicator_data = (
    pd.DataFrame(
        data=np.array([rsi_values, bb_values, mfi_values, cmf_values, targets]).T,
        columns=["rsi", "bb", "mfi", "cmf", "target"],
    )
    .dropna()
    .reset_index(drop=True)
    .astype(np.float64)
)
indicator_data["target"] = indicator_data["target"].astype(np.int32)
indicator_data

,rsi,bb,mfi,cmf,target
0,53.750000,0.834435,47.310164,0.118391,1
1,53.750000,0.979976,56.327443,0.202426,1
2,65.671642,1.006511,60.871871,0.235202,1
3,61.016949,0.910924,67.173006,0.238298,1
4,61.016949,2.210979,74.939408,0.378479,1
...,...,...,...,...,...
2113,57.291667,1.691918,32.142454,-0.255301,1
2114,68.800000,2.966376,47.706655,-0.086727,1
2115,70.895522,2.461380,53.031262,0.006295,0
2116,66.901408,1.604800,51.008568,0.002142,0


In [6]:
indicator_data.describe()

,rsi,bb,mfi,cmf,target
count,2118.000000,2118.000000,2118.000000,2118.000000,2118.000000
mean,49.895664,-0.024507,44.188489,0.098081,0.874410
std,16.466172,1.252253,18.323338,0.243771,0.331465
min,2.247191,-3.465921,1.173070,-0.772300,0.000000
25%,38.160406,-1.015955,30.561603,-0.068747,1.000000
50%,49.593496,-0.053811,43.494468,0.109617,1.000000
75%,61.283057,0.970050,57.039061,0.263337,1.000000
max,96.000000,3.330035,96.698208,0.766618,1.000000


In [7]:
all_data = indicator_data.copy()
all_data[df.columns] = df.iloc[n_steps:, :].values

all_data

,rsi,bb,mfi,cmf,target,open,high,low,close,volume
0,53.750000,0.834435,47.310164,0.118391,1,162.46,162.50,162.45,162.48,156850.0
1,53.750000,0.979976,56.327443,0.202426,1,162.48,162.50,162.48,162.49,83300.0
2,65.671642,1.006511,60.871871,0.235202,1,162.50,162.50,162.49,162.50,52370.0
3,61.016949,0.910924,67.173006,0.238298,1,162.49,162.50,162.49,162.50,28020.0
4,61.016949,2.210979,74.939408,0.378479,1,162.50,162.63,162.49,162.63,62910.0
...,...,...,...,...,...,...,...,...,...,...
2113,57.291667,1.691918,32.142454,-0.255301,1,163.37,163.40,163.30,163.40,171700.0
2114,68.800000,2.966376,47.706655,-0.086727,1,163.40,163.82,163.39,163.71,623050.0
2115,70.895522,2.461380,53.031262,0.006295,0,163.70,163.82,163.61,163.80,314660.0
2116,66.901408,1.604800,51.008568,0.002142,0,163.80,163.82,163.60,163.70,136580.0


# Обработка фичей

In [8]:
scaler = MinMaxScaler()
scaler.fit(indicator_data.iloc[:, :-1])
indicator_data.iloc[:, :-1] = scaler.transform(indicator_data.iloc[:, :-1])
indicator_data

,rsi,bb,mfi,cmf,target
0,0.549347,0.632782,0.482984,0.578777,1
1,0.549347,0.654197,0.577381,0.633384,1
2,0.676507,0.658102,0.624954,0.654682,1
3,0.626859,0.644037,0.690917,0.656694,1
4,0.626859,0.835335,0.772219,0.747785,1
...,...,...,...,...,...
2113,0.587123,0.758957,0.324201,0.335949,1
2114,0.709875,0.946489,0.487134,0.445490,1
2115,0.732227,0.872181,0.542875,0.505936,0
2116,0.689624,0.746138,0.521700,0.503238,0


In [11]:
indicator_data[["rsi", "bb"]].iloc[1].values

array([0.54934684, 0.65419737])

In [9]:
X = indicator_data.iloc[:, :]
y = indicator_data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, shuffle=False
)

In [10]:
step_data = df.iloc[:50]
x = step_data.index
price = step_data["close"]

x_buy = np.array([i for i in x[::2]])
y_buy = price[x_buy]

In [11]:
all_data

,rsi,bb,mfi,cmf,target,open,high,low,close,volume
0,53.750000,0.834435,47.310164,0.118391,1,162.46,162.50,162.45,162.48,156850.0
1,53.750000,0.979976,56.327443,0.202426,1,162.48,162.50,162.48,162.49,83300.0
2,65.671642,1.006511,60.871871,0.235202,1,162.50,162.50,162.49,162.50,52370.0
3,61.016949,0.910924,67.173006,0.238298,1,162.49,162.50,162.49,162.50,28020.0
4,61.016949,2.210979,74.939408,0.378479,1,162.50,162.63,162.49,162.63,62910.0
...,...,...,...,...,...,...,...,...,...,...
2113,57.291667,1.691918,32.142454,-0.255301,1,163.37,163.40,163.30,163.40,171700.0
2114,68.800000,2.966376,47.706655,-0.086727,1,163.40,163.82,163.39,163.71,623050.0
2115,70.895522,2.461380,53.031262,0.006295,0,163.70,163.82,163.61,163.80,314660.0
2116,66.901408,1.604800,51.008568,0.002142,0,163.80,163.82,163.60,163.70,136580.0


In [12]:
indicator_data = indicator_data.iloc[:, :2]

In [16]:
# Your DataFrame `df` should contain all observations and you will have your reward function `reward_function`
# df = all_da.copy()
# df.columns = [col.lower() for col in df.columns]

steps_obs = 3
commision_bps = 0
rewards = []
Statistics = []

init_balance = 200
init_active = 10

start_price = all_data["close"].iloc[0]
init_pv = init_balance + init_active * start_price

MAX_BALANCE = 2 * init_pv
MAX_ACTIVE = 2 * init_pv / start_price


class CustomEnv(gym.Env):
    def __init__(self, df, init_balance, init_active, test=False):
        super(CustomEnv, self).__init__()
        self.df = df
        self.test = test
        self.seed = lambda x: 0
        self.current_step = 0
        # Assume observation space is a 1D array of features (adjust according to your data)
        self.observation_space = spaces.Box(low=0, high=1, shape=(4,), dtype=np.float32)
        self.balance = init_balance
        self.active = init_active
        self.init_balance = init_balance
        self.init_active = init_active
        self.current_price = self.df["close"].iloc[self.current_step]
        self.pv = self.balance + self.active * self.current_price
        # Adjust according to your action space
        self.action_space = spaces.Box(
            low=np.array([-1]),
            high=np.array([1]),
            dtype=np.float32,
        )

    def reset(self):
        # Reset the state of the environment to an initial state
        # print("reseting...")
        self.current_step = 0
        if self.test:
            rewards.append([])
            Statistics.append([])

        init_state = np.append(
            self.df[indicator_data.columns].iloc[0].values,
            np.array([self.init_balance / MAX_BALANCE, self.init_active / MAX_ACTIVE]),
        )
        self.balance = init_balance
        self.active = init_active
        self.current_price = self.df["close"].iloc[self.current_step]
        self.pv = self.balance + self.init_active * self.current_price
        return init_state

    def reward_function(self, action):
        next_price = self.df["close"].iloc[self.current_step + 1]
        # покупаем
        if action > 0:
            money_to_spend = self.balance * action
            self.active += money_to_spend / self.current_price
            self.balance -= money_to_spend
        # продаем
        elif action < 0:
            active_to_sell = self.active * abs(action)
            self.active -= active_to_sell
            self.balance += active_to_sell * self.current_price
        new_pv = self.active * next_price + self.balance
        reward = (new_pv - self.pv) / self.pv * 1e4

        return reward

    def step(self, action):
        # Calculate the reward

        reward = self.reward_function(action)
        self.current_step += 1
        if self.current_step < len(self.df.index) - (steps_obs + 1):
            next_state = np.append(
                self.df[indicator_data.columns].iloc[self.current_step].values,
                np.array([self.balance / MAX_BALANCE, self.active / MAX_ACTIVE]),
            )
            done = False
        else:
            # The episode is finished
            next_state = np.zeros(self.observation_space.shape)
            done = True

        # Placeholder for info, additional data can be added if needed
        info = {}
        # print(next_state)
        if self.test:
            rewards[-1].append(reward)
        self.current_price = self.df["close"].iloc[self.current_step]
        self.pv = self.active * self.current_price + self.balance

        return next_state, reward, done, info


# Create multiple environments
env = CustomEnv(df=all_data, init_balance=init_balance, init_active=init_active)
env = DummyVecEnv([lambda: env])

In [ ]:
# Сохранение и загрузка модели
# model.save("./data/PPO_MultiBinScaling.zip")
# model = PPO.load("./data/PPO_MultiBinScaling.zip", env=env)

In [17]:
policy_kwargs = dict(activation_fn=nn.ReLU, net_arch=dict(pi=[20, 20], vf=[20, 20]))

# Create the PPO model
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    device="cpu",
    policy_kwargs=policy_kwargs,
    learning_rate=0.0001,
    batch_size=128,  # размер одного батча
    n_epochs=5,  # количество повторов на каждом новом наборе батчей
    gamma=0.99,  # степень заглядывания в будущее (от 0 до 0.99999)
    n_steps=128 * 4,  # batch_size x количество батчей
)

# Train the model
model.learn(total_timesteps=2500)  # , progress_bar=True)

# Evaluate the trained model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=2)
print(f"Mean reward: {mean_reward} +/- {std_reward}")

Using cpu device


KeyboardInterrupt: 

In [ ]:
# Reset the environment to get the initial observation
obs = env.reset()

num_iterations = 20
actions_taken = []
rewards = []

for _ in range(num_iterations):
    print(obs)
    # Choose an action randomly (replace this with your own policy)
    action, _ = model.predict(obs, deterministic=False)

    # Take a step in the environment with the predicted action
    obs, reward, done, _ = env.step(action)

    # Save the action taken
    actions_taken.append(action)
    rewards.append(reward)

    if done:
        break

[[42.46575    -0.45067176  0.05483057  0.44516942]]
[[53.94737     0.9793236   0.26998982  0.23001018]]
[[5.3333332e+01 1.0045544e+00 4.6473113e-01 3.5388723e-02]]
[[63.49206     0.6597164   0.          0.49983382]]
[[59.649124    0.850015    0.          0.49983382]]
[[48.88889     0.86629796  0.          0.49983382]]
[[51.11111     0.86253893  0.          0.49983382]]


[[82.14286     0.86253893  0.          0.49983382]]
[[87.17949     2.0831559   0.22073391  0.27926293]]
[[85.29412     1.8733509   0.2286256   0.27138337]]
[[93.75        1.6657321   0.          0.49962947]]
[[96.36364     2.6980762   0.          0.49962947]]
[[82.8125      1.6870966   0.          0.49962947]]
[[86.56716     1.5599154   0.          0.49962947]]
[[76.712326    0.9766236   0.13574617  0.36435813]]
[[69.620255    0.5112316   0.07972115  0.42021456]]
[[71.084335    0.64203644  0.50102264  0.        ]]
[[72.72727     0.7772954   0.32246393  0.17804332]]
[[63.75        0.35981113  0.41902605  0.08178944]]
[[64.19753     0.43969142  0.50105226  0.        ]]


In [ ]:
test_df = all_data.iloc[:num_iterations, :].copy()
test_df["actions"] = np.array(actions_taken).T[0][0]
test_df["rewards"] = np.array(rewards)

test_df

,rsi,bb,target,open,high,low,close,volume,actions,rewards
0,42.465753,-0.450672,1,162.41,162.50,162.37,162.38,102660.0,-0.483320,2.832556
1,53.947368,0.979324,1,162.37,162.50,162.37,162.48,70060.0,-0.846143,0.000000
2,53.333333,1.004554,1,162.48,162.50,162.46,162.48,18360.0,1.000000,-1.229859
3,63.492063,0.659716,1,162.49,162.49,162.39,162.46,39080.0,1.000000,1.230010
4,59.649123,0.850015,1,162.46,162.50,162.45,162.48,156850.0,0.469128,0.616268
5,48.888889,0.866298,1,162.48,162.50,162.48,162.49,83300.0,0.308973,0.614892
6,51.111111,0.862539,1,162.50,162.50,162.49,162.50,52370.0,1.000000,0.000000
7,82.142857,0.862539,1,162.49,162.50,162.49,162.50,28020.0,-0.441288,4.469901
8,87.179487,2.083156,1,162.50,162.63,162.49,162.63,62910.0,-0.028216,0.668079
9,85.294118,1.873351,1,162.62,162.69,162.62,162.65,54210.0,1.000000,0.615207


In [ ]:
fig = px.line(
    test_df, x=test_df.index, y="close", hover_data=["rsi", "bb", "actions", "rewards"]
)
fig.update_traces(mode="markers+lines")

fig.show()